In [ ]:
import glob
import pandas as pd
import numpy as np
import os
import csv
import sys
from pathlib import Path
import random
import datetime

from functools import reduce
from IPython.display import clear_output


In [ ]:
#Read CSV file containing the agents from SPENSER
workplace_destination_employed_selected_dir = r'' # use your path
workplace_destination_employed_selected_file = os.path.join(workplace_destination_employed_selected_dir, "")
df_workplace_destination_employed_selected = pd.read_csv(workplace_destination_employed_selected_file, index_col=None, header=0)

In [ ]:
len(df_workplace_destination_employed_selected)

In [ ]:
df_workplace_destination_employed_selected

In [ ]:
len(msoa_destination_list)

In [ ]:
# Read the buildings used to work
#Read CSV file containing the agents from SPENSER
facilities_location_dir = r'' # use your path
facilities_location_file = os.path.join(facilities_location_dir, "")
df_facilities_location = pd.read_csv(facilities_location_file, index_col=None, header=0)


In [ ]:
len(df_facilities_location)

In [ ]:
df_facilities_location

In [ ]:
## Remove duplicates and keep first
df_facilities_location = df_facilities_location.drop_duplicates(subset='id', keep="first")


In [ ]:
len(df_facilities_location)

In [ ]:
## Remove those rows where msoa11cd is NaN
df_facilities_location = df_facilities_location[df_facilities_location['msoa11cd'].notna()]

In [ ]:
len(df_facilities_location)

In [ ]:
df_facilities_location.dtypes

In [ ]:
df_workplaces_location = df_facilities_location[df_facilities_location['activities'].str.contains('work')]

In [ ]:
len(df_workplaces_location)

In [ ]:
## Save data here:
workplace_building_location = ''

In [ ]:
## Create a new column with the number of square meters for employee in each work category
sqm_per_employee = []
for row in df_workplaces_location['activities']:

    if 'work_office' in row: sqm_per_employee.append(12)
    elif 'work_industrial' in row: sqm_per_employee.append(42)
    elif 'work_retail' in row: sqm_per_employee.append(20)
    elif 'work_commercial' in row: sqm_per_employee.append(50)
    elif 'work_warehouse' in row: sqm_per_employee.append(80)
    elif 'work_hotel' in row: sqm_per_employee.append(70)
    elif 'work_hospital' in row: sqm_per_employee.append(50)
    elif 'work_education' in row: sqm_per_employee.append(30)
    elif 'work_fitness' in row: sqm_per_employee.append(70)
    elif 'work_attraction' in row: sqm_per_employee.append(300)
    elif 'work_restaurant' in row: sqm_per_employee.append(17)
    elif 'work_visitor_attraction' in row: sqm_per_employee.append(50)
    elif 'work_social' in row: sqm_per_employee.append(40)
    elif 'work_entertainment' in row: sqm_per_employee.append(70)
    elif 'work_office' in row: sqm_per_employee.append(12)
    elif 'work_culture' in row: sqm_per_employee.append(200)
    else: sqm_per_employee.append(9999)

In [ ]:
df_workplaces_location['sqm_per_employee'] = sqm_per_employee


In [ ]:
df_workplaces_location['predicted_employees'] = round(df_workplaces_location['floor_area'] / df_workplaces_location['sqm_per_employee'], 0).astype(int)

In [ ]:
df_workplaces_location.head()

In [ ]:
counter = 0

list_tester = []

for msoa_code in msoa_destination_list:
    
    #msoa_code = 'E02002485'
    
    
    temp_list =[]
    
    counter += 1
    clear_output(wait=True)
    print(f' {counter} of {len(msoa_destination_list)}, MSOA name: {msoa_code}')

    # Select those individuals that have been assinged a destination = msoa_code
    df_employed_msoa = df_workplace_destination_employed_selected.loc[(df_workplace_destination_employed_selected['Destination'] == msoa_code)]

    # count the number of people that work there
    total_employed_msoa = len(df_employed_msoa)
    
    #print(total_employed_msoa)
    
    temp_list.append(msoa_code)
    temp_list.append(total_employed_msoa)
    
    # Select those buildings located in the msoa_code
    df_workplaces_location_msoa = df_workplaces_location.loc[(df_workplaces_location['msoa11cd'] == msoa_code)]
    
    number_buildings_msoa = len(df_workplaces_location_msoa)
    
    ## Sum the floor_areas of all buldings in the msoa_code to now the total area
    total_predicted_employees_in_msoa = df_workplaces_location_msoa['predicted_employees'].sum()
    
    #print(total_predicted_employees_in_msoa)
    
  
    employee_ratio = total_employed_msoa / total_predicted_employees_in_msoa

    #print(employee_ratio)
    
    df_workplaces_location_msoa['designed_employees'] = round(df_workplaces_location_msoa['predicted_employees'] * employee_ratio, 0).astype(int)
    
    ## Sum the floor_areas of all buldings in the msoa_code to now the total area
    total_designed_employees_in_msoa = df_workplaces_location_msoa['designed_employees'].sum()
    
    
    #print(total_designed_employees_in_msoa)
    

    
    
    # Sort column 'designed_employees' in ascending order
    ## This will allow start with the building with less employees
    df_workplaces_location_msoa = df_workplaces_location_msoa.sort_values(['designed_employees'], ascending = [True])
    

    building_counter = 0
    number_individuals_msoa_matched_list = 0


    # loop through the buildings
    for idx_building, building_row in df_workplaces_location_msoa.iterrows():
        
        building_counter += 1
        
        if (building_counter != number_buildings_msoa):
            building_employees = building_row['designed_employees']


            if (len(df_employed_msoa) >= building_employees):

                # Select randomly the % of people that has to go to that building
                df_employed_msoa_building_selected = df_employed_msoa.sample(building_employees)

            else:    
                df_employed_msoa_building_selected = df_employed_msoa

            # Remove selected people from the dataframe to avoid selecting them more than once
            ## Concatenate previous selected students with the "potential inactives"
            df_employed_msoa_concatenated = (pd.concat([df_employed_msoa, df_employed_msoa_building_selected]))

            ## Remove duplicates and keep the name of the original dataframe to be used in other iterations:
            df_employed_msoa = df_employed_msoa_concatenated.drop_duplicates(subset='PID_AreaMSOA',keep = False)

        
        
        # Select the remaining people for the last building
        ## This is to avoid leaving people without a building
        else:
            
            df_employed_msoa_building_selected = df_employed_msoa
        

        # Create new columns with informationa about the building selected to them
        df_employed_msoa_building_selected['building_id'] = building_row['id']
        df_employed_msoa_building_selected['x'] = building_row['X']
        df_employed_msoa_building_selected['y'] = building_row['Y']
        
        df_employed_msoa_building_selected['oa'] = building_row['oa11cd']
        df_employed_msoa_building_selected['msoa'] = building_row['msoa11cd']
        
        
        number_individuals_msoa_matched_list += len(df_employed_msoa_building_selected)
        
        # save the data in a csv file
        df_employed_msoa_building_selected.to_csv(workplace_building_location, encoding='utf-8', mode='a', index=False, header=False)
          
        
    temp_list.append(number_individuals_msoa_matched_list)
        
        
    list_tester.append(temp_list)
        
        
        
          

print('Code has been finished. Check results!')   

In [ ]:
list_tester

In [ ]:
df_tester = pd.DataFrame(list_tester)

In [ ]:
df_tester

In [ ]:
df_tester.loc[df_tester[2] == 0]

In [ ]:
df_tester['Difference'] = df_tester[1] - df_tester[2]

In [ ]:
df_tester.loc[(df_tester['Difference'] > 0)]

In [ ]:
df_employed_msoa_building_selected

In [ ]:
header_workplaces_building_location =['PID_AreaMSOA', 'IndividualID', 'DayID', 'Origin', 'Destination', 'building_id', 'x', 'y', 'workplace_oa', 'workplace_msoa']


In [ ]:
#Read CSV file containing the agents from SPENSER
workplaces_building_location_dir = r'' # use your path
workplaces_building_location_file = os.path.join(workplaces_building_location_dir, "")
df_workplaces_building_location = pd.read_csv(workplaces_building_location_file, index_col=None, header=None, names= header_workplaces_building_location)

In [ ]:
len(df_workplaces_building_location)

In [ ]:
df_workplaces_building_location = df_workplaces_building_location.drop_duplicates(subset='PID_AreaMSOA',keep = False)

In [ ]:
df_workplaces_building_location

In [ ]:
len(df_workplaces_building_location)

In [ ]:
len(df_workplace_destination_employed_selected)

In [ ]:
len(df_workplace_destination_employed_selected)- len(df_workplaces_building_location)

In [ ]:
df_workplaces_building_location

In [ ]:
# save the data in a csv file
#df_workplaces_building_location_short.to_csv(r'C:\Users\b9055315\PhD_project\UK_Data_Service\NTS\Generated_data_from_code\workplace_destination\sixth_attempt\workplace_building_location_short.csv', encoding='utf-8', mode='a', index=False, header= header_workplaces_building_location_short)


In [ ]:
buildings_selected_list = df_workplaces_building_location['building_id'].unique().tolist()

In [ ]:
len(buildings_selected_list)

In [ ]:
len(buildings_selected_list)/len(df_workplaces_location)*100